### Imports

In [2]:
import cv2 as cv
import numpy as np
import os

In [3]:
def show_image(title, image):
    image = cv.resize(image, (0, 0), fx=0.2, fy=0.2)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Functia de detectie a conturului

### Functia de modificare a imaginii initiale cu operatii morfologice

In [4]:
### fereastra de proprietati custom
def empty(a):
    pass


def createGUI():
    cv.namedWindow("trackBars", cv.WINDOW_NORMAL)
    # cv.resizeWindow("trackBarks", 640, 240)
    cv.createTrackbar("Hue Min", "trackBars", 20, 179, empty)
    cv.createTrackbar("Hue Max", "trackBars", 140, 179, empty)
    cv.createTrackbar("Saturation Min", "trackBars", 43, 255, empty)
    cv.createTrackbar("Saturation Max", "trackBars", 149, 255, empty)
    cv.createTrackbar("Val Min", "trackBars", 0, 255, empty)
    cv.createTrackbar("Val Max", "trackBars", 255, 255, empty)
    cv.createTrackbar("MedianBlur", "trackBars", 1, 21, empty)
    cv.createTrackbar("GaussianBlur", "trackBars", 1, 21, empty)
    cv.createTrackbar("Sharpen alpha", "trackBars", 1, 20, empty)
    cv.createTrackbar("Sharpen beta", "trackBars", 0, 20, empty)
    cv.createTrackbar("Sharpen gama", "trackBars", 0, 20, empty)
    cv.createTrackbar("tresholdmin", "trackBars", 0, 255, empty)
    cv.createTrackbar("tresholdmax", "trackBars", 255, 255, empty)
    cv.createTrackbar("cannymin", "trackBars", 0, 500, empty)
    cv.createTrackbar("cannymax", "trackBars", 0, 500, empty)


In [5]:
def imgModify(imgOriginala):
    h, w, _ = imgOriginala.shape
    # print(h, w)
    img = cv.resize(imgOriginala, (0, 0), fx=0.2, fy=0.2)

    imgHSV = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    createGUI()
    while True:
        h_min = cv.getTrackbarPos("Hue Min", "trackBars")
        h_max = cv.getTrackbarPos("Hue Max", "trackBars")
        s_min = cv.getTrackbarPos("Saturation Min", "trackBars")
        s_max = cv.getTrackbarPos("Saturation Max", "trackBars")
        v_min = cv.getTrackbarPos("Val Min", "trackBars")
        v_max = cv.getTrackbarPos("Val Max", "trackBars")
        median = cv.getTrackbarPos("MedianBlur", "trackBars")
        gaussian = cv.getTrackbarPos("GaussianBlur", "trackBars")
        alpha = cv.getTrackbarPos("Sharpen alpha", "trackBars")
        beta = cv.getTrackbarPos("Sharpen beta", "trackBars")
        gama = cv.getTrackbarPos("Sharpen gama", "trackBars")
        canny_min = cv.getTrackbarPos("cannymin", "trackBars")
        canny_max = cv.getTrackbarPos("cannymax", "trackBars")
        treshold_min = cv.getTrackbarPos("tresholdmin", "trackBars")
        treshold_max = cv.getTrackbarPos("tresholdmax", "trackBars")

        lower = np.array([h_min, s_min, v_min])
        upper = np.array([h_max, s_max, v_max])
        mask = cv.inRange(imgHSV, lower, upper)

        imgResult = cv.bitwise_and(img, img, mask=mask)

        ###HSV to GRAY
        h, s, v = cv.split(imgResult)
        imgResult = v

        img_m_blur = cv.medianBlur(imgResult, median)
        img_g_blur = cv.GaussianBlur(imgResult, (5, 5), gaussian)
        img_sharpened = cv.addWeighted(img_m_blur, alpha, img_g_blur, beta, gama)
        _, thresh = cv.threshold(img_sharpened, treshold_min, treshold_max, cv.THRESH_BINARY)

        edges = cv.Canny(thresh, canny_min, canny_max)

        contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        img_final = img.copy()
        print(len(contours))
        max_area = 0
        for cont in contours:
            area = cv.contourArea(cont)
            if area > 500:
                peri = cv.arcLength(cont, True)
                cv.drawContours(img_final, cont, -1, (255, 0, 0), 3)
                aprox = cv.approxPolyDP(cont, 0.02 * peri, True)
                x, y, w, h = cv.boundingRect(aprox)
                cv.rectangle(img_final, (x, y), (x + w, y + h), (255, 255, 0), 2)


        cv.imshow("Thresh", thresh)
        cv.imshow("Edges", edges)
        cv.imshow("Original", img)
        cv.imshow("Blur", img_g_blur)
        cv.imshow("Result", img_sharpened)
        cv.imshow("AFISARE PATRAT", img_final)
        cv.waitKey(0)

In [6]:
path = 'antrenare/1_01.jpg'
img = cv.imread(path)

imgModify(img)

# show_image('test', img)

2227
754
2227
2227
2227
2227
2227
581
581
0
581
785
869
1144
1547
1704
1699
1778
1835
1875
1875
1749
1357
0
1175
0
0
1089
1372
1747
1883
1800
1720
1620
1174
675
581
581
581
581
581
581
581
581
581
581
581
581
581
782
801
956
1144
1284
1496
1519
1705
1699
1739
1794
1800
1829
1844
1795
1750
1706
1720
1608
1561
1456
1019
774
717
581
581
581
1611
1618
380
17
17


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\median_blur.dispatch.cpp:285: error: (-215:Assertion failed) (ksize % 2 == 1) && (_src0.dims() <= 2 ) in function 'cv::medianBlur'
